#Bibliotecas e Drives

In [0]:
import pandas as pd
import io
import requests
import folium
from folium import plugins
%matplotlib inline

In [0]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/covid19

/content/drive/My Drive/Colab Notebooks/covid19


In [0]:
!ls

API_SP.POP.TOTL_DS2_en_csv_v2_887275.csv
brasil_20200324.csv
brasil_20200326.csv
Brasil_20200327.html
Brasil_covid19.html
casos.png
covid19Brazil-script.ipynb
covid19-script.ipynb
full_data_20200320.csv
full_data_20200321.csv
full_data_20200322.csv
full_data_20200323.csv
full_data_20200326.csv
Metadata_Country_API_SP.POP.TOTL_DS2_en_csv_v2_887275.csv
Metadata_Indicator_API_SP.POP.TOTL_DS2_en_csv_v2_887275.csv
mortes.png
novos_casos.png
teste.png


#Carregando o banco de dados

In [10]:
url='https://raw.githubusercontent.com/wcota/covid19br/master/cases-gps.csv'
data = requests.get(url).content
brazil_df = pd.read_csv(io.StringIO(data.decode('utf-8')))
brazil_df

,type,name,lat,lon,total
0,1,São Paulo/SP,-23.550651,-46.633382,1044
1,D1,São Paulo/SP,-23.550651,-46.633382,89
2,1,Barra Mansa/RJ,-22.539484,-44.173513,1
3,1,Vila Velha/ES,-20.329729,-40.292028,28
4,1,Feira de Santana/BA,-12.257893,-38.959805,9
...,...,...,...,...,...
367,1,Itaitinga/CE,-3.965768,-38.529763,1
368,1,Maracanaú/CE,-3.877944,-38.624770,1
369,0,NÃO ESPECIFICADA/PI,-7.699278,-42.504379,3
370,0,NÃO ESPECIFICADA/RR,2.135138,-61.363192,4


In [11]:
brazil_df['mortes'] = 0
brazil_df.loc[brazil_df['name'] == 'São Paulo/SP', 'mortes'] = 98
brazil_df.loc[brazil_df['name'] == 'Rio de Janeiro/RJ', 'mortes'] = 17
brazil_df.loc[brazil_df['name'] == 'Salvador/BA', 'mortes'] = 1
brazil_df.loc[brazil_df['name'] == 'Brasília/DF', 'mortes'] = 1
brazil_df.loc[brazil_df['name'] == 'Porto Alegre/RS', 'mortes'] = 2 
brazil_df.loc[brazil_df['name'] == 'Recife/PE', 'mortes'] = 5
brazil_df.loc[brazil_df['name'] == 'Curitiba/PR', 'mortes'] = 2 
brazil_df.loc[brazil_df['name'] == 'Florianópolis/SC', 'mortes'] = 1 
brazil_df.loc[brazil_df['name'] == 'Goiânia/GO', 'mortes'] = 1 
brazil_df.loc[brazil_df['name'] == 'Natal/RN', 'mortes'] = 1 
brazil_df.loc[brazil_df['name'] == 'Manaus/AM', 'mortes'] = 1
brazil_df.loc[brazil_df['name'] == 'Fortaleza/CE', 'mortes'] = 5 
brazil_df.loc[brazil_df['name'] == 'Teresina/PI', 'mortes'] = 1 


brazil_df

,type,name,lat,lon,total,mortes
0,1,São Paulo/SP,-23.550651,-46.633382,1044,98
1,D1,São Paulo/SP,-23.550651,-46.633382,89,98
2,1,Barra Mansa/RJ,-22.539484,-44.173513,1,0
3,1,Vila Velha/ES,-20.329729,-40.292028,28,0
4,1,Feira de Santana/BA,-12.257893,-38.959805,9,0
...,...,...,...,...,...,...
367,1,Itaitinga/CE,-3.965768,-38.529763,1,0
368,1,Maracanaú/CE,-3.877944,-38.624770,1,0
369,0,NÃO ESPECIFICADA/PI,-7.699278,-42.504379,3,0
370,0,NÃO ESPECIFICADA/RR,2.135138,-61.363192,4,0


In [0]:
brazil_df['total'].unique()

array([1044,    1,   28,    9,  516,  298,   12,    6,    7,   39,    2,
        143,   49,   73,   40,   33,   34,  103,  131,    3,   25,   11,
         13,    4,   21,   15,   16,  338,   10,   31,   17,   19,    8,
          5,   18,   14,   27,  231])

## Plotando sobre o mapa


In [0]:
#Definindo o tamanho dos circulos
brazil_df['size']=brazil_df['total'].apply(lambda total:12 if total>=900 else
                                           10 if total>=400 and total<900 else 
                                           8 if total>=200 and total<400 else
                                           6 if total>=50 and total<200 else
                                           4 if total>=20 and total<50 else
                                           2 if total>=10 and total<20 else
                                           0.1)
brazil_df['color']=brazil_df['name'].apply(lambda cidade: 'orange' if 'NÃO' in cidade else
                                           'red')
#plotando o mapa, em PB
mapa = folium.Map(location=[-15.788497,-47.879873],tiles='CartoDB dark_matter', 
                  zoom_start=4)

#Marcação dos pontos
for lat,lon,cidade,total,mortes,size,cor in zip(brazil_df['lat'], brazil_df['lon'],
                                             brazil_df['name'], brazil_df['total'],
                                             brazil_df['mortes'],brazil_df['size'], 
                                             brazil_df['color']):
  texto = "{}<br> Casos: {:,}<br> Mortes: {:,}"
  texto = texto.format(cidade, total, mortes)
  folium.CircleMarker([lat,lon], radius=size, popup=texto, color=cor, 
                      fill=True, fill_opacity=0.9, fill_color='darkred').add_to(mapa)

txt = "Total no Brasil<br>de casos: 4324<br>de mortes: 136<br>data: 29/03/2020"
popup = folium.Popup(txt, max_width=300)
icon = folium.Icon(color='black', icon='info-sign')
marcador = folium.Marker(location=[-15.6914,-25.8008], popup=popup, icon=icon)

mapa.add_child(marcador)

#Mapa de calor
#mapa_calor = brazil_df[['lat', 'lon', 'total']].as_matrix()
#mapa.add_children(plugins.HeatMap(mapa_calor, radius=15))

#mapa.save('Brasil_covid19.html')
mapa